In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Import

In [ ]:
import sys
from logging import INFO, StreamHandler, getLogger

logger = getLogger()
if not logger.hasHandlers():
    logger.addHandler(StreamHandler(sys.stdout))
logger.setLevel(INFO)

In [ ]:
import math
import os
import pathlib

from jinja2 import Environment, FileSystemLoader
from src.four_dim_srda.config.config_loader import load_config
from src.four_dim_srda.config.experiment_config import CFDConfig
from src.four_dim_srda.da.letkf_block_processing import LETKFBlockProcessingConfig

# Define constant

In [ ]:
ROOT_DIR = pathlib.Path(os.environ["PYTHONPATH"]).parent.resolve()

experiment_name = "experiment7"

CFG_DIR = f"{ROOT_DIR}/python/configs/four_dim_srda/{experiment_name}"

cfg_cfd_path = (
    f"{CFG_DIR}/cfd_simulation/qg_model/gpu_make_data_config.yml"
)
CFG_LETKF_DIR = f"{CFG_DIR}/perform_letkf_hr_using_uhr"
os.makedirs(CFG_LETKF_DIR, exist_ok=True)

In [ ]:
SCRIPT_DIR = f"{ROOT_DIR}/python/bash/four_dim_srda/{experiment_name}/letkf_simulation/perform_letkf_hr_using_uhr/es"

SCRIPT_NARROW_JET_DIR = f"{SCRIPT_DIR}/use_narrow_jet"

os.makedirs(SCRIPT_DIR, exist_ok=True)
os.makedirs(SCRIPT_NARROW_JET_DIR, exist_ok=True)

In [ ]:
srda_model_name = "UNetMaxVitVer01"

ogx = 8
ogy = 8

# srda
bea_values = [2]
bed_values = [2]
dspe_values = [360]
nsls_values = [100]
ogx_srda_values = [ogx]
ogy_srda_values = [ogy]

n3drb_values = [3]
nmb_values = [6]
bias_values = [0]

bs_srda_values = [12]
lre_values = ["1e-04"]

list_cfg_srda_name = [
    (
        f"bea{bea}_bed{bed}_dspe{dspe_value}_nsls{nsls}_"
        f"ogx{ogx:02}_ogy{ogy:02}_n3drb{n3drb}_nmb{nmb}_bias{bias}_"
        f"bs{bs:02}_lr{lre}"
    )
    for bea, bed, dspe_value, nsls, ogx, ogy, n3drb, nmb, bias, bs, lre in zip(
        bea_values,
        bed_values,
        dspe_values,
        nsls_values,
        ogx_srda_values,
        ogy_srda_values,
        n3drb_values,
        nmb_values,
        bias_values,
        bs_srda_values,
        lre_values,
    )
]

# config srda
CFG_SRDA_PATH = f"{CFG_DIR}/perform_4D_SRDA/{srda_model_name}/{list_cfg_srda_name[0]}.yml"
cfg_srda = load_config(model_name=srda_model_name, config_path=CFG_SRDA_PATH)

# Define methods

In [ ]:
def format_param(value):
    exponent = int(math.floor(math.log10(abs(value))))
    mantissa = value / (10 ** exponent)
    mantissa_int = int(round(mantissa * 100))
    exponent -= 2

    while mantissa_int % 10 == 0 and mantissa_int != 0:
        mantissa_int //= 10
        exponent += 1

    return f"{mantissa_int}e{exponent:+03d}"

In [ ]:
def make_cfg_and_script(
    *,
    cfg_cfd,
    cfg_letkf,
    cfg_srda,
    experiment_name,
    model_name,
    cfg_letkf_dir,
    script_dir,
    script_narrow_jet_dir,
):
    #
    assert cfg_letkf.obs_grid_interval_x == cfg_srda.dataset_config.obs_grid_interval_x
    assert cfg_letkf.obs_grid_interval_y == cfg_srda.dataset_config.obs_grid_interval_y

    #
    ogx = f"{cfg_letkf.obs_grid_interval_x:02}"
    ogy = f"{cfg_letkf.obs_grid_interval_y:02}"

    # CFD
    ne = f"{cfg_cfd.hr_base_config.n_batch}"
    na = format_param(cfg_cfd.hr_base_config.noise_amplitude)
    cfg_cfd.save(
        pathlib.Path(
            f"{cfg_letkf_dir}/cfd_cfg_ne{ne}_na{na}.yml"
        )
    )
    # LETKF
    ch = format_param(cfg_letkf.cylinder_height)
    cr = format_param(cfg_letkf.cylinder_radius)
    inf_f = format_param(cfg_letkf.inflation_factor)
    lr = format_param(cfg_letkf.localization_radius)
    bs = f"{cfg_letkf.block_size}"

    cfg_letkf.save(
        pathlib.Path(
            f"{cfg_letkf_dir}/letkf_cfg_ogx{ogx}_ogy{ogy}_ne{ne}_ch{ch}_cr{cr}_if{inf_f}_lr{lr}_bs{bs}.yml"
        )
    )

    # SRDA model parameters
    bea = int(cfg_srda.dataset_config.beta_dist_alpha)
    bed = int(cfg_srda.dataset_config.beta_dist_beta)
    dspe = cfg_srda.dataset_config.data_size_per_epoch
    nsls = cfg_srda.dataset_config.num_searched_lr_states

    n3drb = cfg_srda.model_config.num_3d_resnet_block
    nmb = cfg_srda.model_config.num_maxvit_block
    bias = int(cfg_srda.model_config.bias)

    bs_srda = cfg_srda.dataloader_config.batch_size

    lr_srda = format_param(cfg_srda.train_config.lr)

    #
    context = {
        #
        "experiment_name": experiment_name,
        "ogx": ogx,
        "ogy": ogy,

        # CFD
        "ens": ne,
        "na": na,

        # LETKF
        "ch": ch,
        "cr": cr,
        "inf": inf_f,
        "lr": lr,
        "bs": bs,

        # SRDA
        "model_name": model_name,
        #
        "bea": bea,
        "bed": bed,
        "dspe": dspe,
        "nsls": nsls,
        #
        "n3drb": n3drb,
        "nmb": nmb,
        "bias": bias,
        #
        "bs_srda": bs_srda,
        "lr_srda": lr_srda,
    }

    # Set up Jinja2 environment
    env = Environment(
        loader=FileSystemLoader(f"{script_dir}"), trim_blocks=True, lstrip_blocks=True
    )

    # Load and render template
    template = env.get_template("template.sh.j2")
    rendered_script = template.render(context)

    # Generate script
    output_file = f"{script_dir}/na{na}_letkf_cfg_ogx{ogx}_ogy{ogy}_ne{ne}_ch{ch}_cr{cr}_if{inf_f}_lr{lr}_bs{bs}.sh"
    with open(output_file, "w") as f:
        f.write(rendered_script)

    # Grant execution permission
    os.chmod(output_file, 0o740)

    # Set up Jinja2 environment
    env = Environment(
        loader=FileSystemLoader(f"{script_narrow_jet_dir}"), trim_blocks=True, lstrip_blocks=True
    )

    # Load and render narrow jet template
    template = env.get_template("template_using_narrow_jet.sh.j2")
    rendered_script = template.render(context)

    # Generate narrow jet script
    output_file = f"{script_narrow_jet_dir}/na{na}_letkf_cfg_ogx{ogx}_ogy{ogy}_ne{ne}_ch{ch}_cr{cr}_if{inf_f}_lr{lr}_bs{bs}.sh"
    with open(output_file, "w") as f:
        f.write(rendered_script)

    # Grant execution permission
    os.chmod(output_file, 0o740)


# Make

In [ ]:
# CFD

cfg_cfd = CFDConfig.load(pathlib.Path(cfg_cfd_path))
cfg_cfd.hr_base_config.n_batch = 100
cfg_cfd.hr_base_config.noise_amplitude = 3e-3
# logger.info(f"Input config of CFD model = {cfg_cfd.to_json_str()}")

In [ ]:
# LETKF

data_dir = f"{experiment_name}/cfd_simulation/qg_model/hr_pv"
cfg_letkf = LETKFBlockProcessingConfig(
    data_dir=data_dir,
    train_valid_test_kinds="train",
    train_valid_test_ratios=[0.7, 0.2, 0.1],
    num_simulation=1,
    nx=cfg_cfd.hr_base_config.nx,
    ny=cfg_cfd.hr_base_config.ny,
    nz=cfg_cfd.hr_base_config.nz,
    num_ens=cfg_cfd.hr_base_config.n_batch,
    obs_grid_interval_x=ogx,
    obs_grid_interval_y=ogy,
    obs_noise_std=0.022,
    cylinder_radius=6.0,
    cylinder_height=1.6e-3,
    inflation_factor=1.2,
    localization_radius=5.7,
    block_size=6,
)

In [ ]:
make_cfg_and_script(
    cfg_cfd=cfg_cfd,
    cfg_letkf=cfg_letkf,
    cfg_srda=cfg_srda,
    experiment_name=experiment_name,
    model_name=srda_model_name,
    cfg_letkf_dir=CFG_LETKF_DIR,
    script_dir=SCRIPT_DIR,
    script_narrow_jet_dir=SCRIPT_NARROW_JET_DIR,
)